dataassembly.py
Jay Sayre - sayrejay (at) Gmai|, 

Purpose: Combines various datasets and preps code for analysis (see exploratoryanalysis.R script)

Inputs:

Produced by mergingtariffandindustrydata.py:
"cafta-dr/Output/ISICtwodigitleveltariffs.csv" - tariff averages at the ISIC 2-digit level for 2002 and 2013
    
"cafta-dr/Output/municipalityaveragetariff2002.csv" - municipality level tariff averages (for import competing industries, where import competing is considered harmonized system goods that correspond to ISIC
codes based upon conversion table) for the D.R. in 2002, using estimated industrial activity in a municipality for 2002 produced by the script compute_regional_employment.py

"cafta-dr/Output/municipalityaveragetariff2013.csv" - municipality level tariff averages (for import competing industries, where import competing is considered harmonized system goods that correspond to ISIC
codes based upon conversion table) for the D.R. in 2013, using estimated industrial activity in a municipality for 2010 produced by the script compute_regional_employment.py

Produced by DHSdataaggregation.py:
"averageincomebymunicipality2013.csv" - Average income in each D.R. municipality for 2013

"averageincomebyoccmun2013.csv" - Average income in each D.R. municipality and occupation for 2013

Produced by IPUMSdataaggregation.py:
'averageincomebymunicipality2002.csv' - IPUMS 2002 information on the average income of workers in the private sector for a given D.R. municipality

'averageincomebyoccmun2002.csv' - IPUMS 2002 information on the average income of workers in a given ISIC occupation code for a given D.R. municipality

Outputs:

municipality level dataset - TBD

municipality and occupation level dataset - TBD


In [4]:
import pandas as pd
import os

if os.name == 'nt':
    basedir ="D:/Dropbox/Dropbox (Personal)/College/DR_Paper/"
else:
    basedir ="/home/j/Dropbox/College/DR_Paper/"

outputdir = basedir+'cafta-dr/Output/'
    
## INPUTS
isictariffs = outputdir+'ISICtwodigitleveltariffs.csv' #MUN/OCC
muntariff2002 = outputdir+'municipalityaveragetariff2002.csv' #MUN
muntariff2013 = outputdir+'municipalityaveragetariff2013.csv' #MUN
munavginc2002 = outputdir+'averageincomebymunicipality2002.csv' #MUN
munavginc2013 = outputdir+'averageincomebymunicipality2013.csv' #MUN
munavgincocc2002 = outputdir+'averageincomebyoccmun2002.csv' #MUN/OCC
munavgincocc2013 = outputdir+'averageincomebyoccmun2013.csv' #MUN/OCC

## OUTPUTS
#TBD

In [ ]:
### Build municipality level tariff/income data set aka #MUN

In [24]:
### Build municipality and occupation level tariff/income data set aka #MUN/OCC
munoccdf02 = pd.read_csv(munavgincocc2002)
munoccdf13 = pd.read_csv(munavgincocc2013)
isictwodig = pd.read_csv(isictariffs)

munoccdf13.drop(['PROV','MUN'],1,inplace=True)
munoccdf13.columns = ['occ','singresoo','singrthp','sg117a','singresotp','mun']
isictwodig.columns=['occ','duty02','duty08']

munoccdf = munoccdf02.merge(munoccdf13,on=['occ','mun']) ## Need to investigate merge here
munoccdf = munoccdf.merge(isictwodig,on='occ')#,how='left') ## Also here
munoccdf


,occ,mun,inctot,singresoo,singrthp,sg117a,singresotp,duty02,duty08
0,1,101,5869.480427,20000.000000,11233.330000,500.000000,22166.700000,11.192825,2.159741
1,1,101,5869.480427,25000.000000,23625.000000,5000.000000,30000.000000,11.192825,2.159741
2,1,201,2984.791667,4853.235294,3403.495882,1071.333333,5415.470588,11.192825,2.159741
3,1,201,2984.791667,8320.371429,6490.238286,14235.666667,9995.271429,11.192825,2.159741
4,1,202,2500.000000,9030.000000,11030.000000,NaN,11030.000000,11.192825,2.159741
5,1,202,2500.000000,18580.000000,15657.833333,5150.000000,23020.766667,11.192825,2.159741
6,1,203,4800.000000,8604.000000,5053.928000,4725.000000,13500.000000,11.192825,2.159741
7,1,204,3383.333333,5873.333333,4892.963333,4000.000000,8940.000000,11.192825,2.159741
8,1,204,3383.333333,11341.000000,13049.590000,1874.750000,14873.970000,11.192825,2.159741
9,1,205,2500.000000,8083.000000,11933.000000,6600.000000,11383.000000,11.192825,2.159741
